In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape = (150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model =  Model(input = model.input, output = top_model(model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
nb_val_samples=nb_validation_samples)

Model loaded.


/home/nishant/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


/home/nishant/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:78: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/nishant/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:78: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=125, epochs=50, validation_steps=800)`


Epoch 1/50
125/125 [==============================] - 1927s 15s/step - loss: 0.5579 - acc: 0.7265 - val_loss: 0.4917 - val_acc: 0.8025
Epoch 2/50
125/125 [==============================] - 1503s 12s/step - loss: 0.4437 - acc: 0.7880 - val_loss: 0.3801 - val_acc: 0.8725
Epoch 3/50
125/125 [==============================] - 1501s 12s/step - loss: 0.2616 - acc: 0.9005 - val_loss: 0.3444 - val_acc: 0.8900
Epoch 4/50
125/125 [==============================] - 1501s 12s/step - loss: 0.2208 - acc: 0.9185 - val_loss: 0.3138 - val_acc: 0.8975
Epoch 5/50
125/125 [==============================] - 1502s 12s/step - loss: 0.1747 - acc: 0.9335 - val_loss: 0.3515 - val_acc: 0.8888
Epoch 6/50
125/125 [==============================] - 1500s 12s/step - loss: 0.1488 - acc: 0.9425 - val_loss: 0.3460 - val_acc: 0.9012
Epoch 7/50
125/125 [==============================] - 1499s 12s/step - loss: 0.1121 - acc: 0.9545 - val_loss: 0.3678 - val_acc: 0.8988
Epoch 8/50
125/125 [==============================] - 1